In [28]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import os
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
import random
import IPython.display as display
from torchsummary import summary
#from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import cv2
import torchvision.utils as utils
from google.colab.patches import cv2_imshow

import warnings
warnings.filterwarnings("ignore")

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
#Parameters
landscape_photo_dir = '/content/drive/MyDrive/Project Data/gan-getting-started/photo_jpg'
monet_photo_dir = '/content/drive/MyDrive/Project Data/gan-getting-started/monet_jpg'
img_height = 256
img_width = 256
transform = transforms.Compose([transforms.ToTensor()])
learning_rate = 0.001
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 50

In [31]:
#def add_noise(image, noise=0.3):
#  img = transforms.ToTensor()(image)
#  img = img + torch.rand_like(img)*noise
#  img = torch.clip(img,0,1)
#  img = transforms.ToPILImage()(img)
#  return img

#def image_transformations(image):
#  yield transforms.RandomRotation(25,interpolation=Image.BILINEAR,fill=[0.485*255, 0.456*255, 0.406*255])(image)
#  yield transforms.RandomRotation(45,interpolation=Image.BILINEAR,fill=[0.485*255, 0.456*255, 0.406*255])(image)
#  yield transforms.RandomRotation(85,interpolation=Image.BILINEAR,fill=[0.485*255, 0.456*255, 0.406*255])(image)
#  yield transforms.RandomRotation(65,interpolation=Image.BILINEAR,fill=[0.485*255, 0.456*255, 0.406*255])(image)
#  yield add_noise(image,0.25)
#  yield add_noise(image)
#  yield add_noise(image,0.45)
#  yield add_noise(image,0.6)
#  yield transforms.RandomHorizontalFlip(1)(image)
#  yield  transforms.RandomVerticalFlip(1)(image)


#def dataset_augmentation(root_folderpath):
#  files = os.listdir(root_folderpath)
#  for img_path in files:
#    img = Image.open(os.path.join(root_folderpath,img_path))

In [32]:
class ImageDataset(Dataset):

  def __init__(self,root,transform):
    self.root = root
    self.files = os.listdir(root)
    self.transforms = transform

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index):
    image_path = os.path.join(self.root,self.files[index])
    img = Image.open(image_path).convert('RGB')
    img = self.transforms(img)
    return img

In [33]:
class CombineDataset(Dataset):

  def __init__(self):
    self.Image_A = ImageDataset()
    self.Image_B = ImageDataset()

  def __len__(self):
    return min(len(self.Image_A),len(self.Image_B))

  def __getitem__(self, index):
    return self.Image_A[index], self.Image_B[index]

In [34]:
def weights_init():
  "To initialise the weights using different initialisation techniques"
  pass

def set_model():
  "To create an model of given specifications"
  pass

In [35]:
class Generator(nn.Module):

  def __init__(self,in_ch,final_out_ch,filters):
    super().__init__()

    self.e1 = self.encoder_block(in_ch,filters,outermost=True)
    self.e2 = self.encoder_block(filters,filters*2)
    self.e3 = self.encoder_block(filters*2,filters*4)
    self.e4 = self.encoder_block(filters*4,filters*8)
    self.e5 = self.encoder_block(filters*8,filters*8)
    self.e6 = self.encoder_block(filters*8,filters*8)
    self.e7 = self.encoder_block(filters*8,filters*8)
    self.e8 = self.encoder_block(filters*8,filters*8,innermost=True)

    self.d1 = self.decoder_block(filters*8,filters*8)
    self.d2 = self.decoder_block(2*filters*8,filters*8)
    self.d3 = self.decoder_block(2*filters*8,filters*8)
    self.d4 = self.decoder_block(2*filters*8,filters*8)
    self.d5 = self.decoder_block(2*filters*8,filters*4)
    self.d6 = self.decoder_block(2*filters*4,filters*2)
    self.d7 = self.decoder_block(2*filters*2,filters)
    self.d8 = self.decoder_block(2*filters,final_out_ch,outermost=True)

  def encoder_block(self,in_channel,out_channel,innermost=False,outermost=False):
    conv = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,kernel_size=4,stride=2,padding=1)
    activ = nn.LeakyReLU(0.2,True)
    norm = nn.BatchNorm2d(out_channel)
    if outermost:
      model = [conv]
    elif innermost:
      model = [activ, conv]
    else:
      model = [activ, conv, norm]
    return nn.Sequential(*model)

  def decoder_block(self,in_channel,out_channel,outermost=False):
    convtrans = nn.ConvTranspose2d(in_channels=in_channel,out_channels=out_channel,kernel_size=4,stride=2,padding=1)
    activ = nn.ReLU(True)
    norm = nn.Tanh() if outermost else nn.BatchNorm2d(out_channel)
    model = [activ, convtrans, norm]
    return nn.Sequential(*model)

  def forward(self,inputs):
    e1 = self.e1(inputs)
    e2 = self.e2(e1)
    e3 = self.e3(e2)
    e4 = self.e4(e3)
    e5 = self.e5(e4)
    e6 = self.e6(e5)
    e7 = self.e7(e6)
    e8 = self.e8(e7)
    d1 = self.d1(e8)
    d1 = torch.cat([e7,d1],1)
    d2 = self.d2(d1)
    d2 = torch.cat([e6,d2],1)
    d3 = self.d3(d2)
    d3 = torch.cat([e5,d3],1)
    d4 = self.d4(d3)
    d4 = torch.cat([e4,d4],1)
    d5 = self.d5(d4)
    d5 = torch.cat([e3,d5],1)
    d6 = self.d6(d5)
    d6 = torch.cat([e2,d6],1)
    d7 = self.d7(d6)
    d7 = torch.cat([e1,d7],1)
    d8 = self.d8(d7)
    return d8

In [36]:
device = torch.device('cpu')
gen = Generator(3,3,64).to(device)
summary(gen, (3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           3,136
         LeakyReLU-2         [-1, 64, 128, 128]               0
            Conv2d-3          [-1, 128, 64, 64]         131,200
       BatchNorm2d-4          [-1, 128, 64, 64]             256
         LeakyReLU-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 256, 32, 32]         524,544
       BatchNorm2d-7          [-1, 256, 32, 32]             512
         LeakyReLU-8          [-1, 256, 32, 32]               0
            Conv2d-9          [-1, 512, 16, 16]       2,097,664
      BatchNorm2d-10          [-1, 512, 16, 16]           1,024
        LeakyReLU-11          [-1, 512, 16, 16]               0
           Conv2d-12            [-1, 512, 8, 8]       4,194,816
      BatchNorm2d-13            [-1, 512, 8, 8]           1,024
        LeakyReLU-14            [-1, 51

In [37]:
class Discriminator(nn.Module):

  def __init__(self,in_channel,filter):
    super().__init__()
    kernel_size = 4
    stride = 2
    padding = 1
    model = [nn.Conv2d(in_channels=in_channel,out_channels=filter,kernel_size=kernel_size,stride=stride,padding=padding),nn.LeakyReLU(0.2,True)]
    for i in range(3):
      if i == 2:
        stride = 1
      in_scale = 2**i
      out_scale = 2**(i+1)
      model += [
          nn.Conv2d(filter*in_scale,filter*out_scale,kernel_size,stride,padding),
          nn.BatchNorm2d(filter*out_scale),
          nn.LeakyReLU(0.2,True)
      ]
    model += [nn.Conv2d(filter*8,1,kernel_size,1,padding)]
    self.model = nn.Sequential(*model)

  def forward(self,inputs):
    return self.model(inputs)

In [38]:
device = torch.device('cpu')
dis = Discriminator(3,64).to(device)
summary(dis, (3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           3,136
         LeakyReLU-2         [-1, 64, 128, 128]               0
            Conv2d-3          [-1, 128, 64, 64]         131,200
       BatchNorm2d-4          [-1, 128, 64, 64]             256
         LeakyReLU-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 256, 32, 32]         524,544
       BatchNorm2d-7          [-1, 256, 32, 32]             512
         LeakyReLU-8          [-1, 256, 32, 32]               0
            Conv2d-9          [-1, 512, 31, 31]       2,097,664
      BatchNorm2d-10          [-1, 512, 31, 31]           1,024
        LeakyReLU-11          [-1, 512, 31, 31]               0
           Conv2d-12            [-1, 1, 30, 30]           8,193
Total params: 2,766,529
Trainable params: 2,766,529
Non-trainable params: 0
---------------------------

In [42]:
class CycleGanLoss:

  def __init__(self):
    self.classification_loss  = nn.BCEWithLogitsLoss()
    self.reco_loss = nn.L1Loss()

  def discriminator_loss(self,logits,isFake=False):
    target_labels = None
    if isFake:
      target_labels = torch.zeros_like(logits,device=device)
    else:
      target_labels = torch.ones_like(logits,device=device)
    loss = self.classification_loss(logits,target_labels)
    return loss

  def generator_loss(self,logits):
    target_labels = torch.ones_like(logits,device=device)
    loss = self.classification_loss(logits,target_labels)
    return loss

  def reconstruction_loss(self,logits,org):
    loss = self.recon_loss(logits,org)
    return loss

  def __call__(self,real_1,rec_1,real_logits_2,fake_logits_2):
    gen_loss = self.generator_loss(fake_logits_2)
    disc_loss_real = self.discriminator_loss(real_logits_2)
    disc_loss_fake = self.discriminator_loss(fake_logits_2,isFake=True)
    rec_loss = self.reconstruction_loss(rec_1,real_1)
    loss = gen_loss + disc_loss_real + disc_loss_fake + rec_loss
    return loss

In [43]:
class CycleGAN:

  def __init__(self):
    self.Generator_A2B = None
    self.Generator_B2A = None
    self.Discriminator_A = None
    self.Discriminator_B = None
    self.CGLoss = CycleGanLoss()
    self.mode_A2B = True
    self.Gen_A2B_optimizer = torch.optim.Adagrad(
        self.Generator_A2B.parameters(), lr=learning_rate)
    self.Gen_B2A_optimizer = torch.optim.Adagrad(
        self.Generator_B2A.parameters(), lr=learning_rate)
    self.Disc_A_optimizer = torch.optim.Adagrad(
        self.Discriminator_A.parameters(), lr=learning_rate)
    self.Disc_B_optimizer = torch.optim.Adagrad(
        self.Discriminator_B.parameters(), lr=learning_rate)

  def __call__(self,*dataset,Train_mode=False):
    if Train_mode:
      domain_A, domain_B = dataset
      fake_B = self.Generator_A2B(domain_A)
      rec_A = self.Generator_B2A(fake_B)
      fake_A = self.Generator_B2A(domain_B)
      rec_B = self.Generator_A2B(fake_A)
      return fake_B,rec_A,fake_A,rec_B
    else:
      if self.mode_A2B:
        domain_A = dataset[0]
        fake_B = self.Generator_A2B(domain_A)
        return fake_B
      else:
        domain_B = dataset[0]
        fake_A = self.Generator_B2A(domain_B)
        return fake_A

  def train(self,epochs,dataloader):
    for e in epochs:
      for images_A, images_B in dataloader:
        self.Gen_A2B_optimizer.zero_grad()
        self.Gen_B2A_optimizer.zero_grad()
        self.Disc_A_optimizer.zero_grad()
        self.Disc_B_optimizer.zero_grad()
        fake_B, rec_A, fake_A, rec_B = self(images_A,images_B,Train_mode=True)
        fake_logits_A = self.Discriminator_A(fake_A)
        real_logits_A = self.Discriminator_A(images_A)
        fake_logits_B = self.Discriminator_B(fake_B)
        real_logits_B = self.Discriminator_B(images_B)
        A2B_loss = self.CGLoss(images_A,rec_A,real_logits_B,fake_logits_B)
        B2A_loss = self.CGLoss(images_B,rec_B,real_logits_A,fake_logits_A)
        A2B_loss.backward()
        B2A_loss.backward()
        self.Gen_A2B_optimizer.step()
        self.Gen_B2A_optimizer.step()
        self.Disc_A_optimizer.step()
        self.Disc_B_optimizer.step()

  def eval(self):
    pass